# Load Datafile

This notebook is to do an end-to-end loading of a user specified data file and showcase all the 
various resources that gets populated.

#### Pre-requisite
It is assumed that the setup steps are completed successfully. These are:
 - creating the database, schemas, stages
 - defining the external stage
 - defining the functions and stored procedures
 - defining the external tables and views

If you had not done this, use the streamlit (./bin/run_app.sh) to create these as defined in the subpage 'Setup'

In [9]:
# Initialization block
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,VSEKAR,PUBLIC,INDSOL_CMSGOV_PRICING,PUBLIC


In [10]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)

def get_basename_of_datafile(p_datafile:str) -> str:
    base = os.path.basename(p_datafile)
    fl_base = os.path.splitext(base)
    return fl_base[0]


DATA_STAGE = config['APP_DB']['ext_stage']
DATA_STAGE = 'data_stg'
DATA_STAGE_FOLDER = config['APP_DB']['folder_data']
DATA_FILE = '2022_10_01_priority_health_HMO_in-network-rates.zip'
DATA_FILE_BASENAME = get_basename_of_datafile(DATA_FILE)

In [11]:


display(Markdown("### Cleanup block"))
# We will cleanup specific resources and artifacts from possible previous runs.

stmts = [
    f''' truncate table segment_task_execution_status; '''
    ,f''' truncate table task_to_segmentids; '''
    ,f''' truncate table in_network_rates_file_header; '''
    ,f''' truncate table in_network_rates_segment_header; '''
    ,f''' alter stage {DATA_STAGE} refresh; '''
]    
    
print(' truncating tables ...')
for stmt in stmts:
    sp_session.sql(stmt).collect()

print(f''' cleaning up files in external stage under path {DATA_STAGE_FOLDER}/{DATA_FILE_BASENAME}/ ...''')

stmt = f''' select relative_path from directory(@{DATA_STAGE}) where relative_path like '%{DATA_STAGE_FOLDER}/{DATA_FILE_BASENAME}/%'; '''
files = sp_session.sql(stmt).collect()
for r in files:
    stmt = f''' remove @{DATA_STAGE}/{r['RELATIVE_PATH']}; '''
    sp_session.sql(stmt).collect()

### Cleanup block

 truncating tables ...
 cleaning up files in external stage under path data/2022_10_01_priority_health_HMO_in-network-rates/ ...


---
## Data loading
We will be loading the segments and also file header and show case how this functions without the dag.

In [12]:
df = sp_session.call('parse_negotiation_arrangement_segments' ,f'{DATA_STAGE}/{DATA_STAGE_FOLDER}' ,DATA_FILE ,f"@{config['APP_DB']['ext_stage']}/{config['APP_DB']['folder_parsed']}" ,0 ,100)

sp_session.sql(f''' alter stage {DATA_STAGE} refresh; ''').collect()
print(' Status of execution')
print(df)

 Status of execution
{
  "EOF_Reached": false,
  "data_file": "2022_10_01_priority_health_HMO_in-network-rates.zip",
  "elapsed": "=> 0:05:39.496399 ",
  "end_rec_num": 100,
  "last_seg_no": 103,
  "start_rec_num": 0,
  "status": true
}


In [13]:
print(' Parsing file headers')
df = sp_session.call('parse_file_header' ,f'{DATA_STAGE}/{DATA_STAGE_FOLDER}',DATA_FILE)

print(' Status of execution')
print(df)

 Parsing file headers
 Status of execution
{
  "data_file": "2022_10_01_priority_health_HMO_in-network-rates.zip",
  "elapsed": "=> 0:14:30.267876 ",
  "status": true
}


---
## Inspection

In [14]:
display(Markdown("The table in_network_rates_file_header holds the file header elements"))

df = (sp_session.table('in_network_rates_file_header')
        .sample(n=5)
        .to_pandas())

display(df)

The table in_network_rates_file_header holds the file header elements

,DATA_FILE,HEADER,INSERTED_AT
0,2022_10_01_priority_health_HMO_in-network-rates.zip,"{\n ""DATA_FILE"": ""2022_10_01_priority_health_HMO_in-network-rates.zip"",\n ""last_updated_on"": ""2022-10-01"",\n ""reporting_entity_name"": ""Priority Health"",\n ""reporting_entity_type"": ""health insurance issuer"",\n ""total_segments"": 6942,\n ""version"": ""http://json-schema.org/draft-07/schema#""\n}",2022-12-30 17:04:19.565


In [15]:
print(' The HEADER has the following data ')
json.loads(df['HEADER'][0])

 The HEADER has the following data 


{'DATA_FILE': '2022_10_01_priority_health_HMO_in-network-rates.zip',
 'last_updated_on': '2022-10-01',
 'reporting_entity_name': 'Priority Health',
 'reporting_entity_type': 'health insurance issuer',
 'total_segments': 6942,
 'version': 'http://json-schema.org/draft-07/schema#'}

In [16]:
display(Markdown("The table in_network_rates_segment_header lists the various segments that were loaded"))

df = (sp_session.table('in_network_rates_segment_header')
        .sample(n=5)
        .to_pandas())

display(df)

The table in_network_rates_segment_header lists the various segments that were loaded

,DATA_FILE,SEGMENT_ID,NEGOTIATED_RATES_INFO,NEGOTIATED_RATES_COUNT,BUNDLED_CODES_COUNT,COVERED_SERVICES_COUNT,INSERTED_AT
0,2022_10_01_priority_health_HMO_in-network-rates.zip,ffs::cpt::20950::,"{\n ""DATA_FILE"": ""2022_10_01_priority_health_HMO_in-network-rates.zip"",\n ""SEGMENT_ID"": ""ffs::cpt::20950::"",\n ""SEQ_NO"": 23,\n ""billing_code"": ""20950"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Monitor Interstitial Fluid Press Detect"",\n ""name"": ""Monitor Interstitial Fluid Press Detect"",\n ""negotiation_arrangement"": ""ffs""\n}",6385,-1,-1,2022-12-30 16:45:50.792
1,2022_10_01_priority_health_HMO_in-network-rates.zip,ffs::ms-drg::398::,"{\n ""DATA_FILE"": ""2022_10_01_priority_health_HMO_in-network-rates.zip"",\n ""SEGMENT_ID"": ""ffs::ms-drg::398::"",\n ""SEQ_NO"": 77,\n ""billing_code"": ""398"",\n ""billing_code_type"": ""MS-DRG"",\n ""billing_code_type_version"": """",\n ""description"": ""RETICULOENDOTHELIAL & IMMUN DISOR W CC "",\n ""name"": ""RETICULOENDOTHELIAL & IMMUN DISOR W CC"",\n ""negotiation_arrangement"": ""ffs""\n}",3,-1,-1,2022-12-30 16:48:33.686
2,2022_10_01_priority_health_HMO_in-network-rates.zip,ffs::ms-drg::106::,"{\n ""DATA_FILE"": ""2022_10_01_priority_health_HMO_in-network-rates.zip"",\n ""SEGMENT_ID"": ""ffs::ms-drg::106::"",\n ""SEQ_NO"": 66,\n ""billing_code"": ""106"",\n ""billing_code_type"": ""MS-DRG"",\n ""billing_code_type_version"": """",\n ""description"": ""CORONARY BYPASS WITH PTCA "",\n ""name"": ""CORONARY BYPASS WITH PTCA"",\n ""negotiation_arrangement"": ""ffs""\n}",3,-1,-1,2022-12-30 16:47:58.824
3,2022_10_01_priority_health_HMO_in-network-rates.zip,ffs::cpt::67031::,"{\n ""DATA_FILE"": ""2022_10_01_priority_health_HMO_in-network-rates.zip"",\n ""SEGMENT_ID"": ""ffs::cpt::67031::"",\n ""SEQ_NO"": 30,\n ""billing_code"": ""67031"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Laser Destruct Vitreous Lesions"",\n ""name"": ""Laser Destruct Vitreous Lesions"",\n ""negotiation_arrangement"": ""ffs""\n}",5079,-1,-1,2022-12-30 16:46:12.289
4,2022_10_01_priority_health_HMO_in-network-rates.zip,ffs::cpt::27745::,"{\n ""DATA_FILE"": ""2022_10_01_priority_health_HMO_in-network-rates.zip"",\n ""SEGMENT_ID"": ""ffs::cpt::27745::"",\n ""SEQ_NO"": 96,\n ""billing_code"": ""27745"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Prophylactic Tx W/Wo Methylmethacrylate,"",\n ""name"": ""Prophylactic Tx W/Wo Methylmethacrylate,"",\n ""negotiation_arrangement"": ""ffs""\n}",3397,-1,-1,2022-12-30 16:49:29.596


In [17]:
print(' The NEGOTIATED_RATES_INFO has the following data ')
json.loads(df['NEGOTIATED_RATES_INFO'][0])

 The NEGOTIATED_RATES_INFO has the following data 


{'DATA_FILE': '2022_10_01_priority_health_HMO_in-network-rates.zip',
 'SEGMENT_ID': 'ffs::cpt::20950::',
 'SEQ_NO': 23,
 'billing_code': '20950',
 'billing_code_type': 'CPT',
 'billing_code_type_version': '',
 'description': 'Monitor Interstitial Fluid Press Detect',
 'name': 'Monitor Interstitial Fluid Press Detect',
 'negotiation_arrangement': 'ffs'}

In [18]:
para = f''' 
The parsed data are stored as parquet files in the external stage at: @{config['APP_DB']['ext_stage']}/{config['APP_DB']['folder_parsed']}. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.
'''
print(para)

stmt = f''' select relative_path from directory(@{config['APP_DB']['ext_stage']}) where relative_path like '%{config['APP_DB']['folder_parsed']}/reduced_sample_data%' limit 5; '''
df = sp_session.sql(stmt).to_pandas()
display(df)

 
The parsed data are stored as parquet files in the external stage at: @ext_data_stg/raw_parsed. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.



,RELATIVE_PATH
0,raw_parsed/reduced_sample_data/ffs::cpt::15004::/negotiated_rates/data_7_0.parquet.gz
1,raw_parsed/reduced_sample_data/ffs::cpt::15004::/negotiated_rates/data_7_1.parquet.gz
2,raw_parsed/reduced_sample_data/ffs::cpt::33235::/negotiated_rates/data_8_0.parquet.gz
3,raw_parsed/reduced_sample_data/ffs::cpt::33235::/negotiated_rates/data_8_1.parquet.gz
4,raw_parsed/reduced_sample_data/ffs::cpt::62322::/negotiated_rates/data_1_0.parquet.gz


In [19]:
para = f''' 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format
'''
print(para)

pd.set_option('display.max_colwidth', 25)
df = (sp_session.table('ext_negotiated_arrangments_staged')
        .sample(n=5)
        .to_pandas())

display(df)


 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format



,VALUE,P_DATA_FL,P_SEGMENT_ID,P_NEGOTIATION_ARRANGEMENT,P_BILLING_CODE_TYPE,P_BILLING_CODE,P_BILLING_CODE_TYPE_VERSION,P_SEGMENT_TYPE
0,"{\n ""CHUNK_NO"": 1,\n...",reduced_sample_data,ffs::cpt::33235::,ffs,cpt,33235,,negotiated_rates
1,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::76391::,ffs,cpt,76391,,negotiated_rates
2,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::a4614::,ffs,cpt,a4614,,negotiated_rates
3,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::62322::,ffs,cpt,62322,,negotiated_rates
4,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::15004::,ffs,cpt,15004,,negotiated_rates


In [20]:
print(' a sample view of one of the records')
j = json.loads(df['VALUE'][0])
print(j)

 a sample view of one of the records
{'CHUNK_NO': 1, 'DATA_FILE': 'reduced_sample_data.json', 'NEGOTIATED_RATES': [{'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 975.15, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}, {'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 3259.44, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}], 'provider_groups': [{'npi': [1609821081], 'tin': {'type': 'ein', 'value': '382800065'}}]}, {'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 975.15, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}, {'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 3259.44, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}], 'provider_groups': [{'npi': [1427430842], 'tin': {'type': 'ein', 'value': '382800065'}}]}, {'negotiated_

In [21]:
display(Markdown("The table segment_task_execution_status contains the audit of execution for the above stored procedures"))

df = (sp_session.table('segment_task_execution_status')
        .sample(n=5)
        .to_pandas())

display(df)

The table segment_task_execution_status contains the audit of execution for the above stored procedures

,DATA_FILE,TASK_NAME,ELAPSED,TASK_RET_STATUS,INSERTED_AT
0,2022_10_01_priority_h...,,0:05:39.496399,"{""data_file"": ""2022_1...",2022-12-30 16:49:47.074
1,2022_10_01_priority_h...,,0:14:30.267876,"{""data_file"": ""2022_1...",2022-12-30 17:04:19.889


In [22]:
display(Markdown("The view segments_counts_for_datafile_v, based of table segment_task_execution_status, will help to identify the number of negotiated_arrangement segments for a specific data file. This view is populated once all the segments in a specific data files are parsed out"))

df = (sp_session.table('segments_counts_for_datafile_v')
        .sample(n=5)
        .to_pandas())

display(df)

The view segments_counts_for_datafile_v, based of table segment_task_execution_status, will help to identify the number of negotiated_arrangement segments for a specific data file. This view is populated once all the segments in a specific data files are parsed out

,DATA_FILE,TASK_NAME,TOTAL_NO_OF_SEGMENTS


--- 
### Closeout

    With that we are finished this section of the demo setup

In [23]:
sp_session.close()
print('Finished!!!')

Finished!!!
